# Exploracion Api NOAA

## Prueba de API y filtrado de estaciones Mendocinas

 Importar librerías y cargar el Token

In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# Cargamos las variables de entorno desde el archivo .env
load_dotenv()

# Leemos el token de forma segura
TOKEN = os.getenv('NOAA_TOKEN')

# Verificamos que el token se haya cargado correctamente
if not TOKEN:
    print("¡Error! No se encontró el token. Asegúrate de que el archivo .env está en la raíz del proyecto.")
else:
    print("Token cargado exitosamente.")

# Configuramos los datos para las futuras llamadas a la API
headers = {'token': TOKEN}
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"

Token cargado exitosamente.


Obtencion de estaciones meteorologicas de Argentina

In [2]:
# El "endpoint" es la parte de la URL que le dice a la API qué queremos, en este caso, "stations"
endpoint = "stations"

# Preparamos los parámetros para la búsqueda.
# FIPS:AR es el código oficial para Argentina.
# limit=1000 nos trae hasta 1000 resultados de una vez.
params = {
    "locationid": "FIPS:AR",
    "limit": 1000  
}

print("Pidiendo la lista de estaciones a la NOAA...")
# Hacemos la llamada a la API
response = requests.get(base_url + endpoint, headers=headers, params=params)
response.raise_for_status() # Esto verificará si la llamada fue exitosa

print("¡Lista recibida! Creando DataFrame...")
# Convertimos la respuesta (que está en formato JSON) a un DataFrame de pandas
stations_df = pd.DataFrame(response.json()['results'])

# Mostramos las primeras 5 filas para ver cómo se ven los datos
stations_df.head()

Pidiendo la lista de estaciones a la NOAA...
¡Lista recibida! Creando DataFrame...


,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,109.0,1981-01-01,2000-06-01,-29.60,"BAIBENE, AR",1.0,GHCND:AR000000001,METERS,-58.15
1,75.0,1981-01-01,2000-06-01,-29.82,"BONPLAND, AR",1.0,GHCND:AR000000002,METERS,-57.42
2,90.0,1981-01-01,2000-06-01,-29.98,"CAZADORES CORRENTINOS, AR",1.0,GHCND:AR000000003,METERS,-58.28
3,71.0,1981-01-01,2000-06-01,-30.78,"CHAJAR355, AR",1.0,GHCND:AR000000004,METERS,-58.15
4,74.0,1981-01-01,2000-06-01,-30.03,"COLONIA LIBERTAD, AR",1.0,GHCND:AR000000005,METERS,-57.82


Filtramos las estaciones mendocinas

In [3]:
# 1. Definimos el "cajón" geográfico aproximado para la provincia de Mendoza.
#    (Latitud: de sur a norte, Longitud: de oeste a este)
min_lat, max_lat = -37.5, -32.0
min_lon, max_lon = -70.5, -66.5

print(f"Filtrando estaciones dentro del área geográfica de Mendoza...")

# 2. Aplicamos el filtro geográfico.
#    Pedimos todas las filas donde la latitud Y la longitud estén dentro de nuestros límites.
mendoza_stations_geo = stations_df[
    (stations_df['latitude'] >= min_lat) & (stations_df['latitude'] <= max_lat) &
    (stations_df['longitude'] >= min_lon) & (stations_df['longitude'] <= max_lon)
]

# 3. Mostramos el resultado, ordenado por la fecha más reciente de datos.
print("\nEstaciones encontradas en la región de Mendoza (filtrado por coordenadas):")
print(mendoza_stations_geo[['id', 'name', 'mindate', 'maxdate']].sort_values(by='maxdate', ascending=False))

Filtrando estaciones dentro del área geográfica de Mendoza...

Estaciones encontradas en la región de Mendoza (filtrado por coordenadas):
                   id                      name     mindate     maxdate
25  GHCND:AR000087418          MENDOZA AERO, AR  1957-06-30  2025-08-24
58  GHCND:ARM00087416            SAN MARTIN, AR  1973-01-01  2025-08-24
59  GHCND:ARM00087420  MENDOZA OBSERVATORIO, AR  1973-01-01  2025-08-24
66  GHCND:ARM00087506              MALARGUE, AR  1967-06-01  2025-08-24
67  GHCND:ARM00087509            SAN RAFAEL, AR  1973-01-01  2025-08-24


Verificar que las estaciones estén dentro de Mendoza

In [4]:
import folium

# 1. Creamos un mapa base, centrado en Mendoza.
#    Coordenadas aproximadas del centro de la provincia.
mapa_verificacion = folium.Map(location=[-34.6, -68.5], zoom_start=6)

# 2. Añadimos TODAS las estaciones de Argentina al mapa en color azul.
print("Añadiendo todas las estaciones de Argentina al mapa (puntos azules)...")
for index, estacion in stations_df.iterrows():
    folium.CircleMarker(
        location=[estacion['latitude'], estacion['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"{estacion['name']}\nID: {estacion['id']}" # Etiqueta al hacer clic
    ).add_to(mapa_verificacion)

# 3. Añadimos las estaciones filtradas para Mendoza en color rojo y más grandes.
print("Resaltando las estaciones filtradas para Mendoza (puntos rojos)...")
for index, estacion in mendoza_stations_geo.iterrows():
    folium.CircleMarker(
        location=[estacion['latitude'], estacion['longitude']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1.0,
        popup=f"FILTRADA: {estacion['name']}\nID: {estacion['id']}"
    ).add_to(mapa_verificacion)

print("\n¡Mapa generado! Mostrando el mapa a continuación...")

# 4. Mostramos el mapa en la salida de la celda.
mapa_verificacion

Añadiendo todas las estaciones de Argentina al mapa (puntos azules)...
Resaltando las estaciones filtradas para Mendoza (puntos rojos)...

¡Mapa generado! Mostrando el mapa a continuación...


Como podemos observar en el mapa, el box (Cajon) creado para filtrar estaciones de mendoza funciono perfectamente.

Consultar el Estado de la API y peticiones disponibles

## Testeo de descarga de API

In [ ]:
Primero descargamos datos de muestra para probar el funcionamiento de la Api

In [6]:
# --- 1. Definimos los Parámetros de nuestra consulta ---
endpoint = "data"
params = {
    "datasetid": "GHCND",                   # El dataset de resúmenes diarios.
    "stationid": "GHCND:AR000087418",        # ID de la estación MENDOZA AERO.
    "startdate": "2024-01-01",              # Fecha de inicio.
    "enddate": "2024-01-31",                # Fecha de fin (un mes de datos).
    "limit": 1000,                          # Límite de registros por página.
    "units": "metric"                       # Pedimos los datos en unidades métricas (°C, mm).
}

# --- 2. Hacemos la llamada a la API ---
print(f"Pidiendo datos para la estación {params['stationid']}...")
response = requests.get(base_url + endpoint, headers=headers, params=params)
response.raise_for_status() # Verificamos que la llamada fue exitosa.
print("¡Datos recibidos con éxito!")

# --- 3. Procesamos los datos para hacerlos legibles ---
# La API devuelve los datos en formato JSON. Lo convertimos a un DataFrame.
data = response.json().get('results', [])
df_raw = pd.DataFrame(data)

if not df_raw.empty:
    # La API entrega los datos en un formato "largo". Lo "pivotamos" para tener
    # una fila por día y una columna por cada tipo de dato (TMAX, TMIN, etc.)
    df_datos_mes = df_raw.pivot_table(
        index='date', 
        columns='datatype', 
        values='value'
    ).reset_index()

    # Convertimos la columna de fecha a un formato de fecha real.
    df_datos_mes['date'] = pd.to_datetime(df_datos_mes['date'])

    print("\n--- Muestra de Datos Meteorológicos Obtenidos (Enero 2024) ---")
    print(df_datos_mes.head())
else:
    print("\nNo se encontraron datos para el período y estación seleccionados.")

Pidiendo datos para la estación GHCND:AR000087418...
¡Datos recibidos con éxito!

--- Muestra de Datos Meteorológicos Obtenidos (Enero 2024) ---
datatype       date  PRCP  TAVG  TMAX  TMIN
0        2024-01-01   NaN  26.2   NaN   NaN
1        2024-01-02   NaN  27.4   NaN   NaN
2        2024-01-03  13.0  26.2  36.1  16.6
3        2024-01-04   0.0  28.4   NaN  22.8
4        2024-01-05   NaN  29.0  35.8  22.7


## Descarga masiva (bulk)

Descarga masiva de 2000 a 2024 - Con checkpoints por estación descargada

In [10]:
import requests
import pandas as pd
import time
import os
from dotenv import load_dotenv
import calendar

# --- (Configuración Inicial) ---
load_dotenv()
TOKEN = os.getenv('NOAA_TOKEN')
headers = {'token': TOKEN}
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"
print("Preparando la lista de estaciones objetivo...")
estaciones_recientes_df = mendoza_stations_geo[pd.to_datetime(mendoza_stations_geo['maxdate']).dt.year >= 2010].copy()
estaciones_objetivo = pd.Series(estaciones_recientes_df.name.values, index=estaciones_recientes_df.id).to_dict()
print(f"Se procesarán {len(estaciones_objetivo)} estaciones.")
for station_id, station_name in estaciones_objetivo.items():
    print(f"- {station_name} ({station_id})")

# --- Función de Descarga (CON LA LÓGICA DE REINTENTOS REINCORPORADA) ---
def descargar_datos_historicos(station_id, start_year, end_year):
    print(f"\nIniciando descarga para la estación: {station_id}...")
    lista_datos_anuales = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            ultimo_dia_mes = calendar.monthrange(year, month)[1]
            start_date = f"{year}-{str(month).zfill(2)}-01"
            end_date = f"{year}-{str(month).zfill(2)}-{ultimo_dia_mes}"
            params = {"datasetid": "GHCND", "stationid": station_id, "startdate": start_date, "enddate": end_date, "limit": 1000, "units": "metric"}
            
            # === LÓGICA DE REINTENTOS ===
            intentos = 3
            for i in range(intentos):
                try:
                    response = requests.get(base_url + "data", headers=headers, params=params)
                    peticiones_restantes = response.headers.get('X-RateLimit-Remaining', 'N/A')
                    response.raise_for_status()
                    data = response.json().get('results', [])
                    if data:
                        lista_datos_anuales.append(pd.DataFrame(data))
                        print(f"  -> Datos para {month}/{year} OK. (Peticiones restantes: {peticiones_restantes})")
                    else:
                        print(f"  -> Sin datos para {month}/{year}. (Peticiones restantes: {peticiones_restantes})")
                    break 
                except requests.exceptions.HTTPError as e:
                    if e.response.status_code == 503 and i < intentos - 1:
                        print(f"  -> ERROR 503. Reintentando en 10 seg... (Intento {i+1}/{intentos})")
                        time.sleep(10)
                    else:
                        print(f"  -> ERROR al obtener datos para {month}/{year}. Error: {e}")
                        break
                except Exception as e:
                    print(f"  -> ERROR al obtener datos para {month}/{year}. Error: {e}")
                    break
            
            time.sleep(0.5)
            
    if lista_datos_anuales:
        return pd.concat(lista_datos_anuales, ignore_index=True)
    else:
        return pd.DataFrame()

# --- Bucle Principal (con Checkpoints y Reanudación) ---
AÑO_INICIO = 2000
AÑO_FIN = 2024 

for station_id, station_name in estaciones_objetivo.items():
    nombre_archivo_checkpoint = f"../data/raw/station_{station_id.replace(':', '_')}.csv"
    if os.path.exists(nombre_archivo_checkpoint):
        print(f"\nArchivo '{nombre_archivo_checkpoint}' ya existe. Saltando estación {station_name}.")
        continue
    
    datos_estacion = descargar_datos_historicos(station_id, AÑO_INICIO, AÑO_FIN)
    
    if not datos_estacion.empty:
        datos_estacion['station_name'] = station_name
        datos_estacion.to_csv(nombre_archivo_checkpoint, index=False)
        print(f"✅ Checkpoint guardado para {station_name} en '{nombre_archivo_checkpoint}'")

print("\n--- Proceso de Descarga Completado para todas las estaciones ---")

Preparando la lista de estaciones objetivo...
Se procesarán 5 estaciones.
- MENDOZA AERO, AR (GHCND:AR000087418)
- SAN MARTIN, AR (GHCND:ARM00087416)
- MENDOZA OBSERVATORIO, AR (GHCND:ARM00087420)
- MALARGUE, AR (GHCND:ARM00087506)
- SAN RAFAEL, AR (GHCND:ARM00087509)

Archivo '../data/raw/station_GHCND_AR000087418.csv' ya existe. Saltando estación MENDOZA AERO, AR.

Iniciando descarga para la estación: GHCND:ARM00087416...
  -> Datos para 1/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 2/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 3/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 4/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 5/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 6/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 7/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 8/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 9/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 10/2000 OK. (Peticiones restantes: N/A)
  -> Datos para 11/20